In [71]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [72]:
from string import translate, maketrans

In [73]:
home = expanduser("~")
# repertoire=home+"/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/"+annee
repertoire=home+"/Github/Kalaba/Kalaba2/Kanonik/"
repertoire=home+"/pCloud Drive/FOD/GB/Copy/Recherche/Agathe/"
with open(repertoire+"Stems.yaml", 'r') as stream:
    nullStems=yaml.load(stream)
    
noStems=True
prevStems=nullStems.copy()
setRadicals=set()

In [74]:
def randomCVCVC():
    result=""
    result+=choice(list(consonnes), p=consonnesProb)
    result+=rd.choice(voyelles)
    result+=choice(list(consonnes), p=consonnesProb)
    result+=rd.choice(voyelles)
    result+=choice(list(consonnes), p=consonnesProb)
    if result not in setRadicals:
        return result
    else:
        return randomCVCVC()

In [75]:
if noStems:
    for cat in nullStems:
        prevStems[cat]={}
        for lex in nullStems[cat]:
            rdRadical=randomCVCVC()
            prevStems[cat][rdRadical]=nullStems[cat][lex]
            

In [76]:
nomsPropres=["Clemencia","Elphaba","Hoderi","Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["klemensja","elfaba","oderi","kaleb","maira","vjolet","nabil","katiSa","nikol"]

In [77]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [78]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(prevStems)

In [79]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [80]:
with open(repertoire+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)

In [81]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [82]:
consonnes

'ptkbdgmnNfTsSDvzZrljw'

In [83]:
from numpy.random import choice

consonnesFreq=[10,10,10,10,10,10,8,8,6,7,7,7,7,5,5,5,5,8,8,3,3]
consonnesTot=sum(consonnesFreq)
consonnesProb=[float(k)/consonnesTot for k in consonnesFreq]


In [84]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in "iae":
            cibles.append(p)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=phonology["apophonies"][result[cible]]
        return "".join(result)
    else:
        return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [85]:
operations=[devoiserFinale,devoiserFinale,voiserVCV,voiserVCV,changerLongueur,changerLongueur,apophonie,metathese]

In [86]:
def randomRadical(radical):
    result=""
    for letter in radical:
        if letter in voyelles:
            result+=rd.choice(voyelles)
        else:
            result+=choice(list(consonnes), p=consonnesProb)
    print "random",result
    return result

def modifierRadical(radical,dist=[0,1,2,2,3,3,3,4,5]):
    nOperations=rd.choice(dist)
    while nOperations>0:
        operation=rd.choice(operations)
        print nOperations,operation
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    print "modif",radical
    return radical

In [87]:
modifierRadical(randomRadical(u"kaleb"),dist=[0,0,0,1,1,2])

random Zekom
modif Zekom


'Zekom'

In [88]:
racineChanges={}
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(randomRadical(prevLexique[mot]),dist=[0,0,0,1,1,2])
    else:
        print "***"
        print mot,"n'est pas dans le nouveau lexique"
        print "***"

random paZig
modif paZig
random toNet
modif toNet
random mikot
modif mikot
random gevop
modif gevop
random dikeT
modif dikeT
random SukuD
modif SukuD
random fiNem
1 <function devoiserFinale at 0x7f9e1097ccf8>
1 <function devoiserFinale at 0x7f9e1097ccf8>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function changerLongueur at 0x7f9e10931758>
1 <function changerLongueur at 0x7f9e10931758>
1 <function changerLongueur at 0x7f9e10931758>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function metathese at 0x7f9e10931668>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function changerLongueur at 0x7f9e10931758>
modif fiNeme
random watuj
1 <function metathese at 0x7f9e10931668>
modif wajut
random soZeZ
modif soZeZ
random pabov
1 <function changerLongueur at 0x7f9e10931758>
modif pabove
random tulok
1 <function voiserVCV at 0x7f9e109316e0>
1 <function devoiserFinale at 0x7f9e1097ccf8>
1 <function devoiserFinale at 0x7f9e1097ccf8>
1 <function devoiserFinale at 0x7f9e1097ccf8>
1 <function changerLongu

1 <function voiserVCV at 0x7f9e109316e0>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function changerLongueur at 0x7f9e10931758>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function changerLongueur at 0x7f9e10931758>
modif Sonino
random fezez
1 <function changerLongueur at 0x7f9e10931758>
modif fezeza
random kobud
modif kobud
random Sivum
modif Sivum
random naveZ
modif naveZ
random furan
1 <function voiserVCV at 0x7f9e109316e0>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function voiserVCV at 0x7f9e109316e0>
1 <function apophonie at 0x7f9e109319b0>
modif furun
random ziSug
2 <function changerLongueur at 0x7f9e10931758>
2 <function devoiserFinale at 0x7f9e1097ccf8>
1 <function metathese at 0x7f9e10931668>
1 <function changerLongueur at 0x7f9e10931758>
1 <function devoiserFinale at 0x7f9e1097ccf8>
1 <function changerLongueur at 0x7f9e10931758>
1 <function devoiserFin

In [89]:
modifierRadical("Tonif")

3 <function voiserVCV at 0x7f9e109316e0>
3 <function devoiserFinale at 0x7f9e1097ccf8>
3 <function changerLongueur at 0x7f9e10931758>
2 <function changerLongueur at 0x7f9e10931758>
1 <function metathese at 0x7f9e10931668>
modif Tofin


'Tofin'

In [90]:
racineChanges

{'DamaT': 'gepaT',
 'Darep': 'Sonino',
 'Degif': 'gubal',
 'DopoZ': 'fetek',
 'Nanev': 'tadelo',
 'Narif': 'durib',
 'NeduN': 'moNep',
 'Nepok': 'vuTus',
 'Nutem': 'Zetak',
 'Sakan': 'toZune',
 'SaroN': 'maguS',
 'Senar': 'Zolot',
 'Sozap': 'muTib',
 'Supeg': 'gotuz',
 'Supor': 'bizek',
 'Tanop': 'ditet',
 'Tapep': 'retod',
 'Tevet': 'loNot',
 'Tidir': 'lumiT',
 'TikiS': 'segot',
 'ToboD': 'jeTud',
 'Todum': 'fezeza',
 'Tolob': 'pozid',
 'Tudis': 'Sedel',
 'ZaSon': 'furun',
 'ZaZib': 'favet',
 'ZagoD': 'rojob',
 'ZaliT': 'tibefu',
 'Zasur': 'nuTeZ',
 'Zepis': 'nubola',
 'ZiruN': 'dikefa',
 'ZuZaT': 'tudaZ',
 'Zuvaz': 'magok',
 'baDog': 'tinup',
 'baSat': 'kubup',
 'baZoz': 'vekaz',
 'basul': 'gegap',
 'beNaT': 'kides',
 'bejur': 'mutase',
 'beked': 'zuTej',
 'bepiT': 'lugut',
 'bifev': 'sivaS',
 'bilez': 'Sebep',
 'bimuS': 'vipiz',
 'binos': 'buziS',
 'bitef': 'bewep',
 'bobal': 'tapit',
 'buwuf': 'bemud',
 'daZit': 'Tonud',
 'dades': 'Sivum',
 'dadiN': 'taZup',
 'dapiv': 'dobin',
 'de

In [91]:
newStems=replace_keys(prevStems,racineChanges)

In [92]:
with open(repertoire+'randomStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [93]:
with open(repertoire+"randomStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [94]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  NuteS: [en]
  TitiZ: [court]
  Tonud: [verte, vert, verts]
  badiN: [mort]
  bewep: [disparus]
  bubot: [présente]
  bunig: [autres]
  gepaT: [méchants, méchantes, méchante]
  liboro: [enfants, enfant]
  padol: [près]
  runuk: [noire, noires, noir, noirs]
  ruwor: [derniers]
  saDom: [parents]
  savat: [grande, grand]
  zuwoku: [chat]
DET:
  DoSif: [par]
  Nepum: [une]
  SaZaZ: [ses]
  Telig: [entre]
  berok: [sur]
  bozapa: [des]
  daDoT: [au]
  dakog: [six]
  damid: [troisième]
  durib: [à]
  firof: [autres]
  jeTud: [de]
  jiDes: [les]
  kuZir: [en]
  kumel: [aux]
  kused: [pendant]
  lerok: [un]
  minuT: [deux]
  nagud: [quatre]
  nigib: [sept]
  nonan: [trois]
  nubola: [avec]
  razop: [dans]
  retod: [du]
  riSaD: [le]
  sivaS: [près]
  soTer: [vers]
  tudaZ: [pour]
  wetop: [la]
NOM:
  DoDal: [pères]
  Dodor: [lumières, lumière]
  DuguN: [forêt]
  Napej: [compte]
  Nipet: [bras]
  Nokori: [manque]
  Nubif: [esprit]
  Sebep: [place]
  Sedel: [vers]
  Sejok: [chasseur, chas

In [95]:
with open(repertoire+'randomStems.yaml', 'w') as output:
    output.write(yamlText)